The link to all these APIs is: https://www.game-change.co.uk/2023/02/10/a-complete-guide-to-the-fantasy-premier-league-fpl-api/

Descriptions of each dataframe are in the notes.

In [1]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")

Libraries imported successfully......


### i. Add fixture data to the teams data

#### i.i) Expand Team data

a) Teams DF

In [2]:
# Imported full data from last seasons

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()


#Convert to df

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [3]:
def append_event_id(teams_df):
    # Create an empty DataFrame to store the expanded data
    expanded_df = pd.DataFrame()

    # Iterate through events from 1 to 38
    for event_id in range(1, 39):
        # Create a temporary DataFrame for the current event
        temp_df = teams_df.copy()
        
        # Set the 'event_id' column to the current event ID
        temp_df['event_id'] = event_id
        
        # Append the temporary DataFrame to the expanded DataFrame
        expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)
    
    return expanded_df

# Call the function to create an expanded DataFrame with 'event_id'
expanded_teams_df = append_event_id(teams_df)


b) Fixture DF

In [4]:
import requests
import pandas as pd

def fetch_fixture_data(event_id):
    # Define the base URL with the event_id
    base_url = f'https://fantasy.premierleague.com/api/fixtures/?event={event_id}'
    
    # Replace the placeholder '{}' with the actual event_id
    formatted_url = base_url.format(event_id)
    
    # Send an HTTP GET request to the API endpoint
    response = requests.get(formatted_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response to get fixture data
        fixture_json = response.json()
        
        # Create a DataFrame from the fixture data
        fixture_data = pd.DataFrame(fixture_json)
        
        return fixture_data
    else:
        print(f"Request failed for event {event_id} with status code {response.status_code}")
        return None  # Return None in case of a failed request


In [5]:
# Create an empty DataFrame to store the expanded fixture data
expanded_fixture_df = pd.DataFrame()

# Iterate through events from 1 to 38
for event_id in range(1, 39):
# Get fixture data for the current event
    fixture_data = fetch_fixture_data(event_id)
        
    # Create a temporary DataFrame for the fixture data
    temp_df = fixture_data.copy()
        
    # Set the 'event_id' column to the current event ID
    temp_df['event_id'] = event_id
        
    # Append the temporary DataFrame to the expanded fixture DataFrame
    expanded_fixture_df = pd.concat([expanded_fixture_df, temp_df], ignore_index=True)
    
    





In [6]:
# Renaming columns
expanded_teams_df = expanded_teams_df.rename(columns={'name' : 'team_name', 'id':'team_id', 'short_name' : 'team_short_name'})
expanded_fixture_df = expanded_fixture_df.rename(columns={'id':'fixture_id'})

#### i.i) Get fixture id

In [7]:
def get_fixture_id(event_id,team_id):
    for idx, row in expanded_fixture_df.iterrows():
        if event_id == row['event_id']:
            if team_id == row['team_a'] or team_id == row['team_h']:
                return row['fixture_id']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture IDs for all teams in teams_df
expanded_teams_df['fixture_id'] = expanded_teams_df.apply(lambda row: get_fixture_id(row['event_id'], row['team_id']), axis=1)


#### i.ii) Get fixture difficulty

In [8]:
def get_fixture_difficulty(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            if team_id == row['team_a']:
                return row['team_a_difficulty']

            elif team_id == row['team_h']:
                return row['team_h_difficulty']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['fixture_difficulty'] = expanded_teams_df.apply(lambda row: get_fixture_difficulty(row['team_id'], row['fixture_id']), axis=1)


In [9]:
expanded_teams_df.to_csv('expanded_teams_data.csv', index=False)


In [10]:
expanded_teams_df

,code,draw,form,team_id,loss,team_name,played,points,position,team_short_name,...,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id,event_id,fixture_id,fixture_difficulty
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,1230,1285,1250,1250,1210,1320,1,1,2.0,2.0
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,1115,1175,1130,1190,1100,1160,2,1,6.0,4.0
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,1060,1095,1050,1100,1060,1090,127,1,3.0,2.0
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,1125,1205,1120,1220,1130,1190,130,1,8.0,3.0
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,1165,1210,1120,1200,1210,1240,131,1,4.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,17,0,None,16,0,Nott'm Forest,0,0,0,NFO,...,1050,1100,1050,1110,1050,1120,15,38,374.0,2.0
756,49,0,None,17,0,Sheffield Utd,0,0,0,SHU,...,1045,1070,1050,1060,1040,1080,18,38,380.0,3.0
757,6,0,None,18,0,Spurs,0,0,0,TOT,...,1130,1200,1110,1200,1150,1200,21,38,380.0,2.0
758,21,0,None,19,0,West Ham,0,0,0,WHU,...,1095,1140,1100,1150,1090,1130,25,38,379.0,5.0
